In [28]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds

In [29]:
import pprint

We are going to make a dataset called "random" and populate the unique strings in it using the method in the retireval model tutorial

In [30]:
ratings =  tfds.load("movielens/100k-ratings", split = "train")
movies = tfds.load("movielens/100k-movies", split = "train")

In [31]:
for i in movies.take(1).as_numpy_iterator():
    pprint.pprint(i)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


2022-09-18 17:34:43.808489: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [32]:
for i in ratings.take(1).as_numpy_iterator():
    pprint.pprint(i)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


2022-09-18 17:34:43.980231: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [36]:
movies = movies.map(lambda x: {"movie_title" : x["movie_title"]})

ratings = ratings.map(lambda x: {"user_id" : x["user_id"], 
                                "moive_titles" : x["movie_title"]})

KeyError: in user code:

    File "/var/folders/p_/j8lh2l8d3r3dwh95sx4bxlt40000gn/T/ipykernel_11884/1830261322.py", line 3, in None  *
        "moive_titles" : x["movie_title"]}

    KeyError: 'movie_title'


In [35]:
tf.random.set_seed(42)

shuffled = ratings.shuffle(100000, seed = 42, reshuffle_each_iteration = False)

train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)

movie_titles = movies.batch(1000)
user_ids = ratings.batch(1000000).map(lambda x: x["user_id"]) # '<' not supported between instances of 'dict' and 'dict'


unique_movie_titles = np.unique(np.concatenate(list(movie_titles))) # unhashable type: 'list'
unique_user_ids = np.unique(np.concatenate(list(user_ids))) # unhashable type: 'list'



ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
movie_titles

<BatchDataset element_spec={'movie_title': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>